In [258]:
# Clears all of CIRCADIEM databases
mycursor.execute("DROP DATABASE CIRCADIEM;");

In [259]:
import mysql.connector     #python library for connecting to server
import csv
import datetime

import openpyxl
import decimal

connection  = mysql.connector.connect(user='root', password='password123',host='127.0.0.1')

In [260]:
# Create the database and use database
mycursor = connection.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS CIRCADIEM")
connection.database = 'CIRCADIEM'

# Import Functions

In [261]:
# mycursor and connection must be initialized for this function to work. (The correct database should be connected to)
def import_database(table_name, data):
    # Get the headings from the mysql table
    sql_heading_Query = "SHOW COLUMNS FROM {}".format(table_name)
    mycursor.execute(sql_heading_Query)
    heading_data = mycursor.fetchall()
    
    sql_string_heading = "INSERT IGNORE INTO {} (".format(table_name) # INSERT IGNORE is used to avoid writing over previously inserted data points (This will ensure the timestamps are correct)
    for a in range (0, len(heading_data) - 1): # (-1) For the allowance of adding commas to the string
        sql_string_heading = sql_string_heading + heading_data[a][0] + ","
    sql_string_heading = sql_string_heading + "updated_at)"

    for a in range (0, len(data)): #Remove the headings from the dataset 
        sql_string = sql_string_heading + " VALUES ("
        for b in range (0, len(data[0])):
            if data[a][b] == "":
                sql_string = sql_string + 'NULL,'
            elif '"' in str(data[a][b]):
                sql_string = sql_string + '"' + str(data[a][b]).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string = sql_string + '"' + str(data[a][b]) + '",'
        sql_string = sql_string + "NOW())"
        mycursor.execute(sql_string)
    connection.commit()
    
# The function that collapses survey questions as a field rather than a label
def import_database_survey(table_name, data, questions_headings, questions_data):
    # Get the headings from the mysql table
    sql_heading_Query = "SHOW COLUMNS FROM {}".format(table_name)
    mycursor.execute(sql_heading_Query)
    heading_data = mycursor.fetchall()
    
    sql_string_heading = "INSERT IGNORE INTO {} (".format(table_name) # INSERT IGNORE is used to avoid writing over previously inserted data points (This will ensure the timestamps are correct)
    for a in range (0, len(heading_data) - 1): # (-1) For the allowance of adding commas to the string
        sql_string_heading = sql_string_heading + heading_data[a][0] + ","
    sql_string_heading = sql_string_heading + "updated_at)"
    
    for a in range (0, len(data)):
        sql_string = sql_string_heading + " VALUES ("
        for b in range (0, len(data[0])):
            if data[a][b] == "":
                sql_string = sql_string + 'NULL,'
            elif '"' in str(data[a][b]):
                sql_string = sql_string + '"' + str(data[a][b]).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string = sql_string + '"' + str(data[a][b]) + '",'

        for c in range (0, len(questions_data[a])):
            sql_string_collapsed = sql_string

            # Add the question number
            sql_string_collapsed = sql_string_collapsed + '"' + str(c + 1) + '",'
            
            temp_question_heading = questions_headings[c]
            if temp_question_heading == "":
                sql_string_collapsed = sql_string_collapsed + 'NULL,'
            elif '"' in str(temp_question_heading):
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_heading).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_heading) + '",'

            temp_question_answer = questions_data[a][c]
            if temp_question_answer == "":
                sql_string_collapsed = sql_string_collapsed + 'NULL,'
            elif '"' in str(temp_question_answer):
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_answer).replace('"',"'") + '",' #This is to handle strings that contain double quotes. It will replace any double quotes with single quotes
            else:
                sql_string_collapsed = sql_string_collapsed + '"' + str(temp_question_answer) + '",'
            sql_string_collapsed = sql_string_collapsed + "NOW())"
            #print(sql_string_collapsed)
            mycursor.execute(sql_string_collapsed)
    connection.commit()

# CIRCADIEM Hospital

In [262]:
# Export file from Redcap
# Data Exports, Reports, and Stats -> All data (Export data) -> check CSV / Microsoft Excel (labels) export format -> check Remove All Identifier Fields check box De-idification options
# -> Export data

# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2023-08-16_1125.csv"
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital/CIRCADIEMHospital_DATA_LABELS_2023-11-01_1327.csv"


data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

## Create SQL Tables

In [263]:
# Screen
mycursor.execute("""CREATE TABLE IF NOT EXISTS screen (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Record_ID_Hospital_UMRN VARCHAR (8),
Hospital_recruiting_for_CIRCA_DIEM TEXT,
Inborn TEXT,
Sex TEXT, 
Date_and_time_of_birth DATETIME,
DOB DATE,
Expected_date_and_time_of_delivery DATETIME,
EDD DATE,
Gestational_age_w DECIMAL (5,2),
GA_group INTEGER,
GA_Strata TEXT, 
Is_infant_enrolled_in_any_other_interventional_RCT TEXT,
Check_other_RCT_choice_EVARO TEXT,
Check_other_RCT_choice_Protect TEXT,
Check_other_RCT_choice_ProPara TEXT,
Check_other_RCT_choice_PLUSS TEXT,
Check_other_RCT_choice_Cosi2 TEXT,
Check_other_RCT_choice_Provide TEXT,
Check_other_RCT_choice_WASH_T TEXT,
Check_other_RCT_choice_SurfsUp TEXT,
Check_other_RCT_choice_Other TEXT,
Check_other_RCT_choice_No_information TEXT,
Check_other_RCT_choice_Unknown TEXT,
Check_other_RCT_choice_Not_asked TEXT,
Check_other_RCT_choice_Asked_but_unknown TEXT,
Check_other_RCT_choice_Not_applicable TEXT,
Other_Trial TEXT,
Acronym_for_other_RCT TEXT,
Does_the_RCT_have_neurodevelopment_as_its_primary_outcome TEXT,
Does_infant_have_a_congenital_abnormality TEXT,
Please_provide_details_of_congenital_abnormality TEXT,
Is_infant_critically_ill_not_expected_to_survive TEXT,
Expected_discharge_hospital TEXT,
Can_infant_return_for_follow_up_studies_at_2_years TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Main_PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS main_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Have_you_read_the_information_sheet_for_the_CIRCA_DIEM TEXT,
Please_select_the_option_regarding_participation TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Main Econsent
mycursor.execute("""CREATE TABLE IF NOT EXISTS main_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# Main econsent declaration
mycursor.execute("""CREATE TABLE IF NOT EXISTS main_econsent_declaration (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Name_of_Recruiting_PI TEXT,
Signature_of_Recruiting_PI TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Enrol
mycursor.execute("""CREATE TABLE IF NOT EXISTS enrol (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Informed_written_parental_consent TEXT,
Were_parentsguardian_approached_for_consent TEXT,
Why_researcher_unavailable TEXT,
Why_researcher_not_notified_of_admission TEXT,
Why_parents_unable_to_be_contacted_before_72_h TEXT,
Why_parents_dont_understand_English TEXT,
Why_infant_didnt_meet_eligibility_criteria TEXT,
Why_infant_enrolled_in_other_trial TEXT,
Why_Assigned_for_recruit_into_other_trial TEXT,
Why_infant_enrolled_in_maximum_number_of_studies_for_unit TEXT,
Why_other TEXT,
Why_No_information TEXT,
Why_Unknown TEXT,
Why_Not_asked TEXT,
Why_Asked_but_unknown TEXT,
Why_Not_applicable TEXT,
Other_reason_not_approached TEXT,
Type_of_Consent TEXT,
Date_and_time_of_enrol_ DATETIME,
Age_at_enrolment_h DECIMAL (5, 3),
Researcher_obtaining_main_consent TEXT,
Scanned_consent_form_filed TEXT,
Specific_name_of_consent_form_eg_CDXXXXX_mainconsent TEXT,
Consent_form_upload TEXT,
Has_parent_received_a_signed_copy_of_the_consent_form TEXT,
Date_parent_provided_copy_of_consent DATE,
Copy_of_the_consent_form_in_the_infants_medical_record TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Randomise
mycursor.execute("""CREATE TABLE IF NOT EXISTS Randomise (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Is_this_infant_a_twin_or_triplet TEXT,
What_is_the_birth_order_of_this_infant TEXT,
Has_a_twin_or_triplet_already_been_randomised_to_the_study TEXT,
Study_ID_Auto_ID_of_previously_randomised_sibling_twintriplet TEXT,
Eligible_for_Randomisation TEXT,
Confirm_twin_sibling_eligible TEXT,
THIS_infants_randomised_study_group TEXT,
Please_select_the_allocated_group_for_THIS_TWIN TEXT,
Study_Number_from_SITE_PI_ TEXT,
Datetime_of_randomisation DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

# Contacts
mycursor.execute("""CREATE TABLE IF NOT EXISTS contacts (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Is_infant_indigenous TEXT,
Mothers_ethnicity TEXT,
Is_there_a_legal_second_parent_eg_father TEXT,
Second_parent_ethnicity TEXT,
Same_address_as_mother TEXT,
Second_parent_state TEXT,
Alternate_contact_connection_to_parents TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# An_Hx
mycursor.execute("""CREATE TABLE IF NOT EXISTS An_Hx (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Date_of_last_menstrual_period DATE,
Date_of_earliest_antenatal_scan DATE,
Maternal_medical_problems_during_pregnancy TEXT,
Hypertension TEXT,
Asthma TEXT,
Diabetes TEXT,
Hyperthyroidism TEXT,
Hypothyroidism TEXT,
Epilepsy TEXT,
Other TEXT,
No_information TEXT,
Unknown TEXT,
Not_asked TEXT,
Asked_but_unknown TEXT,
Not_applicable TEXT,
Was_the_mother_on_thyroid_medication_during_pregnancy TEXT,
Clinical_chorioamnionitis_ TEXT,
maternal_intrapartum_fever_more_38_C TEXT,
uterine_tenderness TEXT,
maternal_tachycardia TEXT,
fetal_tachycardia TEXT,
foulpurulent_amniotic_fluid TEXT,
choice_not_checked TEXT,
choice_not_recorded TEXT,
choice_No_information TEXT,
choice_Unknown TEXT,
choice_Not_asked TEXT,
choice_Asked_but_unknown TEXT,
choice_Not_applicable TEXT,
Histological_chorioamnionitis TEXT,
Placental_histo_report_filename TEXT,
Placental_histology_report TEXT,
Date_and_time_of_membrane_rupture DATETIME,
ROM_Duration DECIMAL (5, 1),
Antenatal_Steroids TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Birth Hx
mycursor.execute("""CREATE TABLE IF NOT EXISTS birth_hx (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Head_circumference_cm DECIMAL (5, 1),
Birth_weight_g INTEGER,
Birth_weight_Z_score TEXT,
Birthwt_centile TEXT,
IUGR INT,
Delivery_mode TEXT,
APGAR_at_1_min TEXT,
APGAR_at_5_min TEXT,
APGAR_at_10_min TEXT,
Cord_blood_gas TEXT,
Cord_gas_source_use_cord_arterial_sample_if_available TEXT,
Cord_Blood_Gas__pH DECIMAL (10, 3),
Cord_blood_gas__pCO2_mmHg DECIMAL (10, 3),
Cord_blood_gas__BE DECIMAL (10, 3),
Cord_blood_gas__Lactate_mmolL DECIMAL (10, 3),
Meconium_stained_amniotic_fluid TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Preg Surv
mycursor.execute("""CREATE TABLE IF NOT EXISTS preg_surv (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# Infection PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS infection_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Interest_in_CIRCA_DIEM_Infection_and_Immunology_substudy TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Infection eConsent
mycursor.execute("""CREATE TABLE IF NOT EXISTS infection_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# MRI PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS mri_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Interest_in_CIRCA_DIEM_MRI_substudy TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# MRI eConsent
mycursor.execute("""CREATE TABLE IF NOT EXISTS mri_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# Sleep PIS
mycursor.execute("""CREATE TABLE IF NOT EXISTS sleep_pis (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Interest_in_CIRCA_DIEM_sleep_substudy TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Sleep eConsent
mycursor.execute("""CREATE TABLE IF NOT EXISTS sleep_econsent (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# Growth information
mycursor.execute("""CREATE TABLE IF NOT EXISTS growth (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Measurement_date_and_time TEXT,
Corrected_age_at_growth_measurement TEXT,
Crownheel_length TEXT,
Length_centile TEXT,
Length_Z_score TEXT,
Head_circumference TEXT,
HC_centile TEXT,
HC_Z_score TEXT,
Infants_weight INTEGER,
Weight_centile TEXT,
Weight_Z_score TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Pea pod
mycursor.execute("""CREATE TABLE IF NOT EXISTS pea_pod (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Did_infant_have_a_Pea_Pod_assessment_prior_to_discharge TEXT,
No_PeaPod_at_recruitment_hospital TEXT,
Infant_discharged_before_pea_pod_assessment_could_be_arranged TEXT,
no_substudy_consent TEXT,
other_reason TEXT,
No_information TEXT,
Unknown TEXT,
Not_asked TEXT,
Asked_but_unknown TEXT,
Not_applicable TEXT,
PeaPod_test_date_and_time DATETIME,
Peapod_researcher TEXT,
Fat_percent DECIMAL (10, 5),
Fat_free_mass_percent DECIMAL (10, 5),
Fat_mass_in_kg DECIMAL (10, 5),
Fat_free_mass_in_kg DECIMAL (10, 5),
Body_mass_in_kg DECIMAL (10, 5),
Body_volume_in_L DECIMAL (10, 5),
Body_density_ DECIMAL (10, 5),
Fat_mass_density_in_kgL DECIMAL (10, 5),
Fat_free_mass_density_in_kgL DECIMAL (10, 5),
Body_surface_area_in_cm2 DECIMAL (10, 5),
Thoracic_gas_volume_in_L DECIMAL (10, 5),
Peapod_report_filename TEXT,
Peapod_report_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Light Audit
mycursor.execute("""CREATE TABLE IF NOT EXISTS light_audit (
Auto_ID VARCHAR (8),
Event_Name TEXT,
_Was_light_audit_obtained TEXT,
Reason_no_light_monitoring_file_obtained TEXT,
light_audit_date DATE,
Light_log_filename TEXT,
Light_data_log_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Sound Audit
mycursor.execute("""CREATE TABLE IF NOT EXISTS sound_audit (
Auto_ID VARCHAR (8),
Event_Name TEXT,
_Was_sound_audit_obtained TEXT,
Reason_no_sound_file_obtained TEXT,
Sound_audit_date DATE,
Sound_log_filename TEXT,
Sound_log_file_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# 36w_Oximetry_obs
mycursor.execute("""CREATE TABLE IF NOT EXISTS 36w_Oximetry_obs (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Date_36_w_PMA_respiratory_records_obtained TEXT,
Current_respiratory_support_at_36_w_PMA_choice_None TEXT,
Current_respiratory_support_at_36_w_PMA_choice_PBF TEXT,
Current_respiratory_support_at_36_w_PMA_choice_HHF TEXT,
Current_respiratory_support_at_36_w_PMA_choice_CPAP TEXT,
Current_respiratory_support_at_36_w_PMA_choice_MV TEXT,
Current_respiratory_support_at_36_w_PMA_choice_No_information TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Unknown TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Not_asked TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Asked_but_unknown TEXT,
Current_respiratory_support_at_36_w_PMA_choice_Not_applicable TEXT,
O2_flow_in_mLmin TEXT,
HHF_Flow TEXT,
CPAP_Pressure TEXT,
Most_recent_Hb TEXT,
Modified_walsh_oxygen_reduction_air_trial TEXT,
Avge_FiO2__from_shift_test_ TEXT,
Avge_SpO2_from_shift_test INTEGER,
Altitude1 INTEGER,
Altitude2 INTEGER,
File_name_for_24_h_observations TEXT,
24_h_observations TEXT,
Average_SpO2_over_24_h TEXT,
Average_FiO2_over_24_h INTEGER,
Predicted_24_hour_Shift_average_paired_SpO2PiO2 TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Substudies
mycursor.execute("""CREATE TABLE IF NOT EXISTS Substudies (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Biological_substudy_consent TEXT,
Researcher_consenting_biol_substudy TEXT,
Date_of_biol_substudy_consent DATE,
Biol_substudy_consent_filename TEXT,
Biological_substudy_consent_file TEXT,
Physiological_substudy_consent TEXT,
Researcher_consenting_physiology_substudy TEXT,
Date_of_physiology_substudy_consent TEXT,
Physiology_substudy_consent_filename TEXT,
Physiological_substudy_consent_file TEXT,
MRI_substudy_consent TEXT,
Researcher_consenting_MRI_substudy TEXT,
Date_of_MRI_substudy_consent TEXT,
MRI_substudy_consent_filename TEXT,
MRI_substudy_consent_file TEXT,
Sleep_substudy_consent TEXT,
Researcher_consenting_sleep_substudy TEXT,
Date_of_sleep_substudy_consent TEXT,
Sleep_substudy_consent_filename TEXT,
Sleep_substudy_consent_file TEXT,
Social_behaviour_substudy_consent TEXT,
Researcher_consenting_behaviour_substudy TEXT,
Date_of_behaviour_substudy_consent TEXT,
Social_behaviour_substudy_consent_filename TEXT,
Behaviour_substudy_consent_file TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Cardioresp
mycursor.execute("""CREATE TABLE IF NOT EXISTS Cardioresp (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp TEXT,
Main_initial_respiratory_diagnosis TEXT,
Exogenous_surfactant TEXT,
No_of_surfactant_doses TEXT,
Airleak_requiring_drainage TEXT,
Nitric_Oxide_NO_used TEXT,
NO_duration TEXT,
Extracorporeal_Membrane_Oxygenation_ECMO TEXT,
Dur_HHF INTEGER,
Dur_CPAP INTEGER,
Dur_NIV INTEGER,
Dur_IPPV_h INTEGER,
Dur_HFV INTEGER,
Date_last_resp_support DATETIME,
Respiratory_support_at_36w_PMA TEXT,
Systemic_postnatal_steroids_ TEXT,
Hours_of_supplemental_O2 INTEGER,
PDA TEXT,
Type_of_treatment_for_PDA_choice_None_or_fluid_restrict TEXT,
Type_of_treatment_for_PDA_choice_pharmacological TEXT,
Type_of_treatment_for_PDA_choice_surgery TEXT,
Type_of_treatment_for_PDA_choice_No_information TEXT,
Type_of_treatment_for_PDA_choice_Unknown TEXT,
Type_of_treatment_for_PDA_choice_Not_asked TEXT,
Type_of_treatment_for_PDA_choice_Asked_but_unknown TEXT,
Type_of_treatment_for_PDA_choice_Not_applicable TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Headus
mycursor.execute("""CREATE TABLE IF NOT EXISTS Headus (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Head_ultrasound_done TEXT,
Date_of_Head_Ultrasound DATETIME,
Maximum_grade_of_left_sided_periventricular_haemorrhage TEXT,
Maximum_grade_of_right_sided_periventricular_haemorrhage TEXT,
Cerebral_cysts TEXT,
Cerebral_cysts_Left TEXT,
Cerebral_cysts_Right TEXT,
File_name_for_USS_report TEXT,
Ultrasound_report TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# MRI
mycursor.execute("""CREATE TABLE IF NOT EXISTS MRI (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Was_near_term_MRI_done TEXT,
Date_and_time_of_pre_dischargenear_term_MRI DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

# Neomorbidity
mycursor.execute("""CREATE TABLE IF NOT EXISTS Neomorbidity (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Was_NEC_diagnosed TEXT,
Datetime_of_NEC_diagnosis TEXT,
NEC_treatment_choice_none TEXT,
NEC_treatment_choice_medical TEXT,
NEC_treatment_choice_surgery TEXT,
NEC_treatment_choice_No_information TEXT,
NEC_treatment_choice_Unknown TEXT,
NEC_treatment_choice_Not_asked TEXT,
NEC_treatment_choice_Asked_but_unknown TEXT,
NEC_treatment_choice_Not_applicable TEXT,
NEC_stage TEXT,
TPN_initiated TEXT,
TPN_hours INTEGER,
TPN_days INTEGER,
Datetime_TPN_ceased DATETIME,
Date_achieved_enteral_feed_120_mLkg TEXT,
Breast_milk_feeding_at_discharge TEXT,
Worst_ROP TEXT,
ROP_Plus TEXT,
Surgical_ROP_Rx TEXT,
VEGF TEXT,
Culture_ve_sepsis TEXT,
No_of_culture_ve_sepsis_episodes TEXT,
Phototherapy_for_Jaundice TEXT,
Maximum_SBR INTEGER,
Max_SBR_date_and_time TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# discharge
mycursor.execute("""CREATE TABLE IF NOT EXISTS discharge (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Survey_Timestamp DATETIME,
Datetime_of_discharge_from_primary_recruiting_hospital DATETIME,
Where_was_infant_discharge_to_from_the_primary_recruiting_site TEXT,
Secondary_hospital TEXT,
Other_hospital_Secondary TEXT,
Where_was_infant_discharged_to_after_2nd_hospital TEXT,
Date_and_time_infant_discharged_from_second_hospital DATETIME,
Third_hospital TEXT,
Other_hospital_Third TEXT,
Where_was_infant_discharged_to_after_3rd_hospital TEXT,
Date_and_time_infant_discharged_from_third_hospital DATETIME,
Fourth_hospital TEXT,
Other_hospital_Fourth TEXT,
Where_was_infant_discharged_to_after_4th_hospital TEXT,
Date_and_time_infant_discharged_from_fourth_hospital DATETIME,
Fifth_hospital TEXT,
Other_hospital_Fifth TEXT,
Date_and_time_infant_discharged_from_fifth_hospital DATETIME,
Date_and_time_infant_discharged_home DATETIME,
Duration_of_hospitalisation_Days DECIMAL (5, 1),
Discharge_comment TEXT,
O2_ceased_in_hospital TEXT,
Datetime_O2_ceased TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Survival
mycursor.execute("""CREATE TABLE IF NOT EXISTS survival (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Has_infant_died TEXT,
Date_and_time_of_death_ DATETIME,
Primary_cause_of_death TEXT,
postnatal_age_at_time_of_death TEXT,
PMA_at_time_of_death TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Adverse Events
mycursor.execute("""CREATE TABLE IF NOT EXISTS adverse_event (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Was_there_an_adverse_event_related_to_eye_mask_or_ear_plugs TEXT,
DateTime_of_AE_noted TEXT,
Study_device_involved_in_AE_choice_eye_mask TEXT,
Study_device_involved_in_AE_choice_ear_plugs TEXT,
Study_device_involved_in_AE_choice_No_information TEXT,
Study_device_involved_in_AE_choice_Unknown TEXT,
Study_device_involved_in_AE_choice_Not_asked TEXT,
Study_device_involved_in_AE_choice_Asked_but_unknown TEXT,
Study_device_involved_in_AE_choice_Not_applicable TEXT,
AE_Type_choice_skin_damage_from_use_of_eye_mask_or_ear_plugs TEXT,
AE_Type_choice_infection_associated_with_eye_mask_or_ear_plugs TEXT,
AE_Type_choice_safety_issue_with_eye_mask_or_ear_plugs TEXT,
AE_Type_choice_No_information TEXT,
AE_Type_choice_Unknown TEXT,
AE_Type_choice_Not_asked TEXT,
AE_Type_choice_Asked_but_unknown TEXT,
AE_Type_choice_Not_applicable TEXT,
Was_AE_life_threatening TEXT,
Describe_skin_damage TEXT,
Describe_infection TEXT,
Describe_safety_issue TEXT,
Was_there_any_other_adverse_event TEXT,
Describe_other_adverse_event TEXT,
AE_MedDRA_coding_term_1 TEXT,
AE_MedDRA_coding_term_2 TEXT,
AE_MedDRA_coding_term_3 TEXT,
Was_other_adverse_event_life_threatening TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Protocol Deviation
mycursor.execute("""CREATE TABLE IF NOT EXISTS protocol_deviation (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Was_there_a_protocol_deviation_recorded_for_this_participant TEXT,
Date_of_deviation DATE,
Date_identified DATE,
Deviation_description TEXT,
Deviation_Code__ TEXT,
Deviation_Code_Other TEXT,
Resulted_in_Adverse_Event TEXT,
Did_participant_continue_in_study TEXT,
Date_of_withdrawal DATE,
Ethics_Reporting_required TEXT,
Ethics_report_date TEXT,
PI_signature TEXT,
Protocol_deviation_form_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Withdrawal
mycursor.execute("""CREATE TABLE IF NOT EXISTS withdrawal (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Study_withdrawal TEXT,
Withdrawal_datetime DATETIME,
Withdrawal_form TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# Report Calculations
mycursor.execute("""CREATE TABLE IF NOT EXISTS report_calculations (
Auto_ID VARCHAR (8),
Event_Name TEXT,
Hospital_duration_primary_days_Duration DECIMAL (5,1),
Secondary_hospital_duration_days DECIMAL (5,1),
Interim_secondary_hospital_site__  TEXT,
Third_hospital_duration_days DECIMAL (5,1),
Interim_third_hospital_site__  TEXT,
Fourth_hospital_duration_days DECIMAL (5,1),
Interim_fourth_hospital_site TEXT,
Hospital_duration_primary_days_Duration_until_discharged_home DECIMAL (5,1),
Discharge_instruments_complete INTEGER,
Survival INTEGER,
Withdraw INTEGER,
Complete TEXT,
updated_at TIMESTAMP)""")

## Import Data

In [264]:
# Initial 6 columns refer to linking fields
# ï»¿"Auto_ID"
# Event_Name
# Repeat_Instrument
# Repeat_Instance
# Data_Access_Group
# Survey_Identifier

# Identify the Columns for each Instrument
Hospital_Instruments = ["screen", "main_pis", "main_econsent", "main_econsent_declaration", "enrol", "randomise", "contacts", "an_hx", "birth_hx", "preg_surv", "infection_pis", "infection_econsent", "mri_pis", "mri_econsent", "sleep_pis", "sleep_econsent", "growth", "pea_pod", "light_audit", "sound_audit", "36w_oximetry_obs", "substudies", "cardioresp", "headus", "mri", "neomorbidity", "discharge", "survival", "adverse_event", "protocol_deviation", "withdrawal", "report_calculations"]

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [6] # Starting after the last instrument
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if temp == "Complete?": # This is the last column for the Instrument
        Column_list.append(a + 1)

event_name_list = ["D0-3", "D7", "D14", "D28", "D56", "36w", "Disch", "Protocol Deviation", "Study Withdrawal", "Survival"]  

# Manually create list for each event type that triggers an instrument
D03 = ["screen", "main_pis", "main_econsent", "main_econsent_declaration", "enrol", "randomise", "contacts", "an_hx", "birth_hx", "preg_surv", "growth", "headus", "report_calculations"]
D7 = ["infection_pis", "infection_econsent", "growth", "headus"]
D14 = ["growth"]
D28 = ["growth", "headus"]
D56 = ["growth"]
W36 = ["infection_pis", "infection_econsent", "mri_pis", "mri_econsent", "sleep_pis", "sleep_econsent", "growth", "pea_pod", "36w_oximetry_obs", "headus"]
Disch = ["growth", "substudies", "cardioresp", "mri", "neomorbidity", "discharge"]
Protocol_deviation = ["protocol_deviation"]
Study_withdrawal = ["withdrawal"]
Survival = ["survival"]
         
event_name_mapping = [D03, D7, D14, D28, D56, W36, Disch, Protocol_deviation, Study_withdrawal, Survival]
       
# The name of the filter for the repeat Instruments
event_name_list_repeatInstrument = ["Light Audit", "Sound Audit", "AdverseEvent"]

# Create a dictionary with the Hospital Instruments / If it is a survey / Starting Column / Ending Column / Event to Instrument Mapping / Repeat_Instruments

# Tables are stored into the database based on whether they are a survey or not (Manually determined)
# 1 if Survey
# 0 if Not Survey

# Manually deduce whether or not the instrument should be stored in a survey structure
Hospital_Instruments_Survey = ["main_econsent", "preg_surv", "infection_econsent", "mri_econsent", "sleep_econsent"]

# The datarows are filtered either based on event_name or repeat_instrument. This will display the right data row when the data should be present
# 1 if filtered by Repeat Instrument
# 0 if filtered by Event Name

# Data rows that need to be filtered by repeat_instruments rather than event_name
Repeat_Instruments = ["light_audit", "sound_audit", "adverse_event"]

Hospital_Dictionary = {}
# Example screen: [0, 6, 41] -> instrument: "screen", not survey, first column: 6, last column: 41
for a in range (0, len(Hospital_Instruments)):
    table_name = Hospital_Instruments[a]
    
    temp_array = []
    # Check to see if the instrument is a survey
    if table_name in Hospital_Instruments_Survey:
         temp_array.append(1)
    else:
         temp_array.append(0)
         
    # First column and Last column of data
    temp_array.append(Column_list[a])
    temp_array.append(Column_list[a+1])
         
    # Check to see if the instrument needs to be filtered by Repeat Instrument
    if table_name in Repeat_Instruments:
        temp_array.append(1)
         # Append an array of the repeat_instrument strings that it need to be filtered on
        temp_list = []
        for b in range (0, len(Repeat_Instruments)):
            if Repeat_Instruments[b] == table_name:
                temp_list.append(event_name_list_repeatInstrument[b])
        temp_array.append(temp_list)
    else:
        temp_array.append(0)
        # Append an array of the event_names that need to be filtered on
        temp_list = []
        for b in range (0, len(event_name_mapping)):
            if table_name in event_name_mapping[b]:
                temp_list.append(event_name_list[b])
        temp_array.append(temp_list)
        
    # Create Dictionary
    Hospital_Dictionary[table_name] = temp_array

In [265]:
Hospital_Dictionary

{'screen': [0, 6, 41, 0, ['D0-3']],
 'main_pis': [0, 41, 45, 0, ['D0-3']],
 'main_econsent': [1, 45, 65, 0, ['D0-3']],
 'main_econsent_declaration': [0, 65, 68, 0, ['D0-3']],
 'enrol': [0, 68, 96, 0, ['D0-3']],
 'randomise': [0, 96, 107, 0, ['D0-3']],
 'contacts': [0, 107, 115, 0, ['D0-3']],
 'an_hx': [0, 115, 151, 0, ['D0-3']],
 'birth_hx': [0, 151, 168, 0, ['D0-3']],
 'preg_surv': [1, 168, 190, 0, ['D0-3']],
 'infection_pis': [0, 190, 193, 0, ['D7', '36w']],
 'infection_econsent': [1, 193, 211, 0, ['D7', '36w']],
 'mri_pis': [0, 211, 214, 0, ['36w']],
 'mri_econsent': [1, 214, 224, 0, ['36w']],
 'sleep_pis': [0, 224, 227, 0, ['36w']],
 'sleep_econsent': [1, 227, 240, 0, ['36w']],
 'growth': [0,
  240,
  253,
  0,
  ['D0-3', 'D7', 'D14', 'D28', 'D56', '36w', 'Disch']],
 'pea_pod': [0, 253, 279, 0, ['36w']],
 'light_audit': [0, 279, 285, 1, ['Light Audit']],
 'sound_audit': [0, 285, 291, 1, ['Sound Audit']],
 '36w_oximetry_obs': [0, 291, 317, 0, ['36w']],
 'substudies': [0, 317, 343, 0

In [266]:
# Iterate through the dictionary of Instruments, Check to see if they are survey / Check to see what the rows are filtered on / Import to SQL

for table_name in Hospital_Dictionary:
    
    #print(table_name)
    
    Survey_binary = Hospital_Dictionary[table_name][0]
    Column1 = Hospital_Dictionary[table_name][1]
    Column2 = Hospital_Dictionary[table_name][2]
    Repeat_Instrument_bin = Hospital_Dictionary[table_name][3]
    StringList = Hospital_Dictionary[table_name][4]
    
    
    # If the Instrument is NOT a survey
    if Hospital_Dictionary[table_name][0] == 0:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
            else:
                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
                
        # Check to see if the final columne heading is "Comlete?"
        if data[0][Column2 - 1] != "Complete?":
            print("Error: Last Column was not 'Complete?' field")

        # Import the data into the correct table in the database
        import_database(table_name, import_data)
        print("{} imported".format(table_name))
        
    # If the Instrument is a survey
    if Hospital_Dictionary[table_name][0] == 1:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        questions_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)
            else:
                if event_name in StringList and repeat_instrument == "":
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)

            # Gather the headings of questions of the survey
            questions_headings = data[0][Column1+1:Column2-1] 

        # Impot survey structured data into the database
        import_database_survey(table_name, import_data, questions_headings, questions_data)
        print("{} imported".format(table_name))

screen imported
main_pis imported
main_econsent imported
main_econsent_declaration imported
enrol imported
randomise imported
contacts imported
an_hx imported
birth_hx imported
preg_surv imported
infection_pis imported
infection_econsent imported
mri_pis imported
mri_econsent imported
sleep_pis imported
sleep_econsent imported
growth imported
pea_pod imported
light_audit imported
sound_audit imported
36w_oximetry_obs imported
substudies imported
cardioresp imported
headus imported
mri imported
neomorbidity imported
discharge imported
survival imported
adverse_event imported
protocol_deviation imported
withdrawal imported
report_calculations imported


In [267]:
# View all the headings of the the fields separated by the "Complete" field for different forms
# for a in range (Column1,Column2):
#     temp = data[0][a].replace(" ", "_").replace("(","").replace(")","").replace("?","").replace("=", "_").replace("+", "").replace(".", "").replace("/", "").replace("-", "_").replace(":", "")
#     print(a, temp)
    
#     if temp == "Complete":
#         print("--------------------------------------------------------------------------------")

# CIRCADIEM FollowUp

In [268]:
# Export file from Redcap
# Data Exports, Reports, and Stats -> All data (Export data) -> check CSV / Microsoft Excel (labels) export format -> Export data

# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_FU/CIRCADIEMFU_DATA_LABELS_2023-09-27_0929.csv"

data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)

## Create SQL Tables

In [269]:
# Core Details
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_core_details (
Study_ID TEXT,
Event_Name TEXT,
Repeat_Instrument TEXT,
Repeat_Instance TEXT,
Data_Access_Group TEXT,
Survey_Identifier TEXT,
Record_ID_Hospital_UMRN TEXT,
Hospital_recruiting_for_CIRCA_DIEM TEXT,
Sex TEXT,
DOB DATE,
EDD DATE,
GA_Strata TEXT,
Check_other_RCT_choice_EVARO TEXT,
Check_other_RCT_choice_Protect TEXT,
Check_other_RCT_choice_ProPara TEXT,
Check_other_RCT_choice_PLUSS TEXT,
Check_other_RCT_choice_Cosi2 TEXT,
Check_other_RCT_choice_Provide TEXT,
Check_other_RCT_choice_WASHT TEXT,
Check_other_RCT_choice_SurfsUp TEXT,
Check_other_RCT_choice_Other TEXT,
Check_other_RCT_choice_No_information TEXT,
Check_other_RCT_choice_Unknown TEXT,
Check_other_RCT_choice_Not_asked TEXT,
Check_other_RCT_choice_Asked_but_unknown TEXT,
Check_other_RCT_choice_Not_applicable TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_contacts
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_contacts (
Study_ID TEXT,
Event_Name TEXT,
Infant_First_Name TEXT,
Infants_surname_in_hospital TEXT,
Infant_Last_Name TEXT,
Is_infant_indigenous TEXT,
Mothers_medical_record_number TEXT,
Mothers_first_Name TEXT,
Mothers_Surname TEXT,
Mothers_ethnicity TEXT,
UnitNumber_and_Street TEXT,
Suburb TEXT,
State TEXT,
Post_Code TEXT,
Mothers_Home_Phone_Number TEXT,
Mothers_Mobile_PhNo TEXT,
Mothers_Email TEXT,
Second_Parent_Preferred_First_Name TEXT,
Second_Parent_Last_Name TEXT,
Second_Parent_Ethnicity TEXT,
Second_Parent_Address TEXT,
Second_Parent___FlatNumber_Street_Address TEXT,
Second_Parent_Suburb TEXT,
Second_Parent_State TEXT,
Second_Parent_Post_Code TEXT,
Second_Parent_Home_Phone_Number TEXT,
Second_Parent_Mobile_PhNo TEXT,
Second_Parent_Email TEXT,
Alternate_contact_preferred_first_name TEXT,
Alternate_contact_last_name TEXT,
Alternate_contact_connection_to_parents TEXT,
Alternate_contact_Mobile_PhNo TEXT,
Infants_paediatrician_First_Name TEXT,
Infants_paediatrician_Surname TEXT,
First_name_of_Infants_GP TEXT,
Surname_of_infants_GP TEXT,
GPs_clinic_name TEXT,
Number_and_street_of_GP_practice TEXT,
GP_Suburb TEXT,
GP_Postcode TEXT,
GP_Contact_Number TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_growth
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_growth (
Study_ID TEXT,
Event_Name TEXT,
Measurement_date_and_time DATETIME,
Corrected_age_at_growth_measurement INTEGER,
Crownheel_length DECIMAL (5,1),
Length_centile  DECIMAL (5,1),
Length_Z_score DECIMAL (5,1),
Head_circumference DECIMAL (5,1),
HC_centile DECIMAL (5,1),
HC_Z_score DECIMAL (5,1),
Infants_weight INTEGER,
Weight_centile DECIMAL (5,1),
Weight_Z_score DECIMAL (5,1),
Complete TEXT,
updated_at TIMESTAMP)""")

# FU pregnancy survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_preg_surv (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Date_pregnancy_survey_started DATE,
Have_you_already_completed_this_survey_for_sibling TEXT,
During_the_last_month_of_pregnancy_bed_time TIME,
During_the_last_month_of_pregnancy_wake_time TIME,
Maximum_sleep_time DECIMAL (5, 1),
How_would_you_rate_the_quality_of_your_sleep TEXT,
Did_your_occupation_require_you_to_do_any_shift_work TEXT,
Did_your_shift_work_involve_overnight_shifts TEXT,
Typical_start_time_for_overnight_shifts TEXT,
Typical_time_overnight_shifts_ended TEXT,
Typical_number_of_night_shifts_per_week TEXT,
Did_you_stop_overnight_shift_work_during_pregnancy TEXT,
About_when_did_you_stop_shift_work_in_pregnancy TEXT,
Comments_about_your_shiftwork_during_pregnancy TEXT,
Have_you_taken_nicotine_substance TEXT,
Did_you_take_other_nicotine_substance_during_pregnancy TEXT,
During_your_pregnancy_how_many_cigarettes_per_day TEXT,
Did_you_stop_smoking_before_the_end_of_your_pregnancy TEXT,
How_many_weeks_pregnant_were_you_when_you_stopped_smoking TEXT,
Mother_basic_health_and_lifestyle_questionnaire_upload_ TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_substudies
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_substudies (
Study_ID TEXT,
Event_Name TEXT,
Infant_Sleep_Substudy TEXT,
Sleep_substudy_consent_filename TEXT,
MRI_substudy_consent TEXT,
Name_of_MRI_Substudy_Consent_Form__assumed_pdf TEXT,
MRI_substudy_consent_file TEXT,
Autism_substudy_consent TEXT,
Name_of_Autism_Substudy_Consent_Form__assumed_pdf TEXT,
Autism_substudy_consent_form TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_survival
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_survival (
Study_ID TEXT,
Event_Name TEXT,
Has_infant_died TEXT,
Date_of_death_ DATE,
Primary_cause_of_death TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_2m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_2m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_6m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_6m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_12m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_12m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_24m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_24m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_27m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_27m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_30m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_30m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_33m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_33m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Asq3_36m
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_Asq3_36m (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

#FU_ASQ3_count
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_asq_score (
Study_ID TEXT,
Event_Name TEXT,
ASQ3_2m_communication_count INT,
ASQ3_2m_gross_motor_count INT,
ASQ3_2m_fine_motor_count INT,
ASQ3_2m_problem_solving_count INT,
ASQ3_2m_personal_social_count INT,
ASQ3_2m_communication_subscore DECIMAL (10,7),
ASQ3_2m_gross_motor_subscore DECIMAL (10,7),
ASQ3_2m_fine_motor_subscore DECIMAL (10,7),
ASQ3_2m_problem_solving_subscore DECIMAL (10,7),
ASQ3_2m_personal_social_subscore DECIMAL (10,7),
ASQ3_2m_total_score DECIMAL (10,7),
ASQ3_6m_communication_count INT,
ASQ3_6m_gross_motor_count INT,
ASQ3_6m_fine_motor_count INT,
ASQ3_6m_problem_solving_count INT,
ASQ3_6m_personal_social_count INT,
ASQ3_6m_communication_subscore DECIMAL (10,7),
ASQ3_6m_gross_motor_subscore DECIMAL (10,7),
ASQ3_6m_fine_motor_subscore DECIMAL (10,7),
ASQ3_6m_problem_solving_subscore DECIMAL (10,7),
ASQ3_6m_personal_social_subscore DECIMAL (10,7),
ASQ3_6m_total_score DECIMAL (10,7),
ASQ3_12m_communication_count INT,
ASQ3_12m_gross_motor_count INT,
ASQ3_12m_fine_motor_count INT,
ASQ3_12m_problem_solving_count INT,
ASQ3_12m_personal_social_count INT,
ASQ3_12m_communication_subscore DECIMAL (10,7),
ASQ3_12m_gross_motor_subscore DECIMAL (10,7),
ASQ3_12m_fine_motor_subscore DECIMAL (10,7),
ASQ3_12m_problem_solving_subscore DECIMAL (10,7),
ASQ3_12m_personal_social_subscore DECIMAL (10,7),
ASQ3_12m_total_score DECIMAL (10,7),
ASQ3_24m_communication_count INT,
ASQ3_24m_gross_motor_count INT,
ASQ3_24m_fine_motor_count INT,
ASQ3_24m_problem_solving_count INT,
ASQ3_24m_personal_social_count INT,
ASQ3_24m_communication_subscore DECIMAL (10,7),
ASQ3_24m_gross_motor_subscore DECIMAL (10,7),
ASQ3_24m_fine_motor_subscore DECIMAL (10,7),
ASQ3_24m_problem_solving_subscore DECIMAL (10,7),
ASQ3_24m_personal_social_subscore DECIMAL (10,7),
ASQ3_24m_total_score DECIMAL (10,7),
ASQ3_Trak_2m_communication_count INT,
ASQ3_Trak_2m_gross_motor_count INT,
ASQ3_Trak_2m_fine_motor_count INT,
ASQ3_Trak_2m_problem_solving_count INT,
ASQ3_Trak_2m_personal_social_count INT,
ASQ3_Trak_2m_communication_subscore DECIMAL (10,7),
ASQ3_Trak_2m_gross_motor_subscore DECIMAL (10,7),
ASQ3_Trak_2m_fine_motor_subscore DECIMAL (10,7),
ASQ3_Trak_2m_problem_solving_subscore DECIMAL (10,7),
ASQ3_Trak_2m_personal_social_subscore DECIMAL (10,7),
ASQ3_Trak_2m_total_score DECIMAL (10,7),
ASQ3_Trak_6m_communication_count INT,
ASQ3_Trak_6m_gross_motor_count INT,
ASQ3_Trak_6m_fine_motor_count INT,
ASQ3_Trak_6m_problem_solving_count INT,
ASQ3_Trak_6m_personal_social_count INT,
ASQ3_Trak_6m_communication_subscore DECIMAL (10,7),
ASQ3_Trak_6m_gross_motor_subscore DECIMAL (10,7),
ASQ3_Trak_6m_fine_motor_subscore DECIMAL (10,7),
ASQ3_Trak_6m_problem_solving_subscore DECIMAL (10,7),
ASQ3_Trak_6m_personal_social_subscore DECIMAL (10,7),
ASQ3_Trak_6m_total_score DECIMAL (10,7),
ASQ3_Trak_12m_communication_count INT,
ASQ3_Trak_12m_gross_motor_count INT,
ASQ3_Trak_12m_fine_motor_count INT,
ASQ3_Trak_12m_problem_solving_count INT,
ASQ3_Trak_12m_personal_social_count INT,
ASQ3_Trak_12m_communication_subscore DECIMAL (10,7),
ASQ3_Trak_12m_gross_motor_subscore DECIMAL (10,7),
ASQ3_Trak_12m_fine_motor_subscore DECIMAL (10,7),
ASQ3_Trak_12m_problem_solving_subscore DECIMAL (10,7),
ASQ3_Trak_12m_personal_social_subscore DECIMAL (10,7),
ASQ3_Trak_12m_total_score DECIMAL (10,7),
ASQ3_Trak_24m_communication_count INT,
ASQ3_Trak_24m_gross_motor_count INT,
ASQ3_Trak_24m_fine_motor_count INT,
ASQ3_Trak_24m_problem_solving_count INT,
ASQ3_Trak_24m_personal_social_count INT,
ASQ3_Trak_24m_communication_subscore DECIMAL (10,7),
ASQ3_Trak_24m_gross_motor_subscore DECIMAL (10,7),
ASQ3_Trak_24m_fine_motor_subscore DECIMAL (10,7),
ASQ3_Trak_24m_problem_solving_subscore DECIMAL (10,7),
ASQ3_Trak_24m_personal_social_subscore DECIMAL (10,7),
ASQ3_Trak_24m_total_score DECIMAL (10,7),
ASQ3_notes TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_ITSEA_survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_itsea (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_ITSEA_score
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_itsea_score (
Study_ID TEXT,
Event_Name TEXT,
ActivityImpulsivity_subscale_count INT,
ActivityImpulsivity_subscale_score DECIMAL (10,7),
Agressiondefiance_subscale_count INT,
Aggressiondefiance_subscale_score DECIMAL (10,7),
Peer_agression_subscale_count INT,
Peer_aggression_subscale_score DECIMAL (10,7),
Externalising_Total_Score DECIMAL (10,7),
Depressionwithdrawal_subscale_count INT,
Depressionwithdrawal_subscale_score DECIMAL (10,7),
Fear_composite_count INT,
Fear_Composite_Score DECIMAL (10,7),
General_anxiety_subscale_count INT,
General_anxiety_subscale_score DECIMAL (10,7),
Separationdistress_subscale_count INT,
Separation_distress_subscale_score DECIMAL (10,7),
Inhibition_to_novelty_subscale_count INT,
Inhibition_to_novelty_subscale_score DECIMAL (10,7),
Internalising_total_score DECIMAL (10,7),
Negative_emotionality_subscale_count INT,
Negative_emotionality_subscale_score DECIMAL (10,7),
Sleep_subscale_count INT,
Sleep_subscale_score DECIMAL (10,7),
Eating_subscale_count INT,
Eating_subscale_score DECIMAL (10,7),
Sensory_sensitivity_subscale_count INT,
Sensory_sensitivity_subscale_score DECIMAL (10,7),
Dysregulation_Total_Score DECIMAL (10,7),
Compliance_subscale_count INT,
Compliance_subscale_score DECIMAL (10,7),
Attention_subscale_count INT,
Attention_subscale_score DECIMAL (10,7),
Mastery_motivation_subscale_count INT,
Mastery_motivation_subscale_score DECIMAL (10,7),
Imitationplay_subscale_count INT,
Imitationplay_subscale_score DECIMAL (10,7),
Empathy_subscale_count INT,
Empathy_subscale_score DECIMAL (10,7),
Prosocial_peer_relations_subscale_count INT,
Prosocial_peer_relations_subscale_score DECIMAL (10,7),
Competence_Total_Score DECIMAL (10,7),
Maladaptive_item_cluster_subscale_count INT,
Maladaptive_item_cluster_subscale_score DECIMAL (10,7),
Social_relatedness_item_cluster_subscale_count INT,
Social_relatedness_item_cluster_subscale_score DECIMAL (10,7),
Atypical_item_cluster_subscale_count INT,
Atypical_item_cluster_subscale_score DECIMAL (10,7),
Individual_items_of_clinical_significance_score DECIMAL (10,7),
ASD_Problem_BItsea INT,
ASD_Competence_Bitsea INT,
ASD_Problem_Hill INT,
ASD_Competence_Hill INT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_CSBS_checklist
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_CSBS (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

#FU_ITSEA_score
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_csbs_score (
Study_ID TEXT,
Event_Name TEXT,
Emotion_and_Eye_Gaze INT,
Communication INT,
Gestures INT,
Sounds INT,
Words INT,
Understanding INT,
Object_Use INT,
Social_Composite INT,
Speech_Composite INT,
Symbolic_Composite INT,
Total_Score_Composite INT,
Social_Composite_Std_Score DECIMAL (10,7),
Speech_Composite_Std_Score DECIMAL (10,7),
Symbolic_Composite_Std_Score DECIMAL (10,7),
Total_Score_Standard_Score DECIMAL (10,7),
Social_Composite_Percentile_Rank DECIMAL (10,7),
Speech_Composite_Percentile_Rank DECIMAL (10,7),
Symbolic_Composite_Percentile_Rank DECIMAL (10,7),
Total_Score_Percentile_Rank DECIMAL (10,7),
CSBS_notes TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_FFM
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_FFM (
Study_ID TEXT,
Event_Name TEXT,
Date_of_FFM DATE,
Fat_free_mass_kg DECIMAL(5,2),
Percentage_body_fat DECIMAL (5,2),
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_bayley4
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_bayley4 (
Study_ID TEXT,
Event_Name TEXT,
B4_assess_date DATE,
B4_absolute_age_months INT,
Corrected_Age_months INT,
Corrected_Age_days INT,
Assessor_Name TEXT,
Assessment_Location TEXT,
Other_assessment_location TEXT,
Child_successfully_tested TEXT,
If_no_why TEXT,
Specify_reason_for_failure TEXT,
Failed_other_reason TEXT,
Specify_reason_for_declining TEXT,
Decline_other_reason TEXT,
Specify_reason_for_loss_to_follow_up TEXT,
Specify_other_reason TEXT,
Assessment_rebooked TEXT,
Assessment_administered_in_childs_primary_language_ TEXT,
Bayley4_report_name TEXT,
Bayley4_report_upload TEXT,
Total_raw_score INT,
Scaled_score INT,
Composite_Score INT,
Receptive_Language_raw_score INT,
Receptive_Language_scaled_score INT,
Expressive_Language_raw_score INT,
Expressive_Language_scaled_score INT,
Sum_of_Scaled_Scores1 INT,
Composite_Score1 INT,
Fine_Motor_raw_score INT,
_Fine_Motor_scaled_score INT,
Gross_Motor_raw_score INT,
Gross_Motor_scaled_score INT,
Sum_of_Scaled_Scores2 INT,
Composite_Score2 INT,
Notes_on_any_missing_data TEXT,
Parent_friendly_Bayley4_report_requested TEXT,
Date_parent_friendly_Bayley4_report_requested DATE,
Parent_friendly_Bayley4_report_sent TEXT,
Date_parent_friendly_Bayley4_report_sent DATE,
Complete TEXT,
updated_at TIMESTAMP)""")

#FU_adverse_event
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_adverse_event (
Study_ID TEXT,
Event_Name TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_No TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_Yes TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_No_information TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_Unknown TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_Not_asked TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_Asked_but_unknown TEXT,
AE_related_to_eye_mask_or_ear_plugs_choice_Not_applicable TEXT,
DateTime_of_AE_noted DATETIME,
Study_device_involved_in_AE_choice_eye_mask TEXT,
Study_device_involved_in_AE_choice_ear_plugs TEXT,
Study_device_involved_in_AE_choice_No_information TEXT,
Study_device_involved_in_AE_choice_Unknown TEXT,
Study_device_involved_in_AE_choice_Not_asked TEXT,
Study_device_involved_in_AE_choice_Asked_but_unknown TEXT,
Study_device_involved_in_AE_choice_Not_applicable TEXT,
AE_skin_damage_from_use_of_eye_mask_or_ear_plugs TEXT,
AE_infection_associated_with_eye_mask_or_ear_plugs TEXT,
AE_safety_issue_associated_with_eye_mask_or_ear_plugs TEXT,
AE_No_information TEXT,
AE_Unknown TEXT,
AE_Not_asked TEXT,
AE_Asked_but_unknown TEXT,
AE_Not_applicable TEXT,
Was_AE_life_threatening TEXT,
Describe_skin_damage TEXT,
Describe_infection TEXT,
Describe_safety_issue TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_study_withdrawal
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_withdrawal (
Study_ID TEXT,
Event_Name TEXT,
Study_withdrawal TEXT,
Withdrawal_datetime DATETIME,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_M_Int_Survey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_M_Int_Survey (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_2ySurvey
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_y2_Surv (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_actigraphy_child
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_actigraphy_child (
Study_ID TEXT,
Event_Name TEXT,
Date_actiwatch_applied DATE,
Time_actiwatch_applied TIME,
Date_actiwatch_removed DATE,
Time_actiwatch_removed TIME,
Actigraphy_cPNA_Months DECIMAL (5,2),
Watch_location TEXT,
Actigraphy_log_filename TEXT,
Actigraphy_file_upload TEXT,
Was_24_hours_of_recording_available TEXT,
Total_sleep_24_hours DECIMAL (5,2),
Total_overnight_sleep_7pm_7am DECIMAL (5,2),
Longest_sleep_epoch_overnight DECIMAL (5,2),
Total_daytime_sleep_7_am___7_pm DECIMAL (5,2),
Longest_daytime_sleep DECIMAL (5,2),
Daytime_activity_ratio DECIMAL (5,2),
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_actigraphy maternal
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_actigraphy_maternal (
Study_ID TEXT,
Event_Name TEXT,
Date_actiwatch_applied DATE,
Time_actiwatch_applied TIME,
Date_actiwatch_removed DATE,
Time_actiwatch_removed TIME,
Watch_location TEXT,
Actigraphy_log_filename TEXT,
Actigraphy_file_upload TEXT,
Was_24_hours_of_recording_available TEXT,
Total_sleep_24_hours DECIMAL (5,2),
Total_overnight_sleep_7pm_7am DECIMAL (5,2),
Longest_sleep_epoch_overnight DECIMAL (5,2),
Total_daytime_sleep_7_am___7_pm DECIMAL (5,2),
Longest_daytime_sleep DECIMAL (5,2),
Daytime_activity_ratio DECIMAL (5,2),
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_sample_collection
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_salivary_swabs_collection (
Study_ID TEXT,
Event_Name TEXT,
Collection_datetime_morning DATETIME,
Saliva_am_volume_uL INTEGER,
Melatonin_am_pgmLL INTEGER,
Collection_datetime_late_evening DATETIME,
Saliva_pm_volume_uLL INTEGER,
Melatonin_pm_pgmLL INTEGER,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_BISQR
mycursor.execute("""CREATE TABLE IF NOT EXISTS fu_bisqr (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Complete TEXT,
Question_number INT,
Question TEXT,
Result TEXT,
updated_at TIMESTAMP)""")

# FU_Sleep_diary
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_sleep_diary (
Study_ID TEXT,
Event_Name TEXT,
Survey_Timestamp DATETIME,
Sleep_diary_start_date DATETIME,
Sleep_Diary_pdf_upload TEXT,
Day_Sleep_time DATETIME,
Day_Wake_time DATETIME,
Day_Last_night_I_fell_asleep TEXT,
Day_Number_of_times_woken_up INTEGER,
Day_Number_of_minutes_woken_up INTEGER,
Day_Sleep_hours DECIMAL (5,2),
Day_Sleep_was_disturbed_by TEXT, 
Day_When_I_woke_up_for_the_day_I_felt TEXT, 
Day_Sleep_notes TEXT,
Eve_I_consumed_caffeinated_drinks_in_Morning TEXT,
Eve_I_consumed_caffeinated_drinks_in_Afternoon TEXT,
Eve_I_consumed_caffeinated_drinks_in_Evening TEXT,
Eve_I_consumed_caffeinated_drinks_in_Not_Applicable1 TEXT,
Eve_I_consumed_caffeinated_drinks_in_No_information TEXT,
Eve_I_consumed_caffeinated_drinks_in_Unknown TEXT,
Eve_I_consumed_caffeinated_drinks_in_Not_asked TEXT,
Eve_I_consumed_caffeinated_drinks_in_askedbutunknown TEXT,
Eve_I_consumed_caffeinated_drinks_in_Not_applicable2 TEXT,
Eve_How_many INTEGER,
Eve_I_exercised_at_least_20_minutes_Morning TEXT,
Eve_I_exercised_at_least_20_minutes_Afternoon TEXT,
Eve_I_exercised_at_least_20_minutes_Evening TEXT,
Eve_I_exercised_at_least_20_minutes_Not_Applicable1 TEXT,
Eve_I_exercised_at_least_20_minutes_No_information TEXT,
Eve_I_exercised_at_least_20_minutes_Unknown TEXT,
Eve_I_exercised_at_least_20_minutes_Not_asked TEXT,
Eve_I_exercised_at_least_20_minutes_askedbutunknown TEXT,
Eve_I_exercised_at_least_20_minutes_Not_applicable2 TEXT,
Eve_Medications_I_took_today TEXT,
Eve_Took_a_nap TEXT,
Eve_If_Yes_for_how_long INTEGER,
Eve_During_the_day_how_likely_was_I_to_doze_off_daily_activities TEXT,
Eve_Throughout_the_day_my_mood_was TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_Alcohol TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_A_heavy_meal TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_Caffeine TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_Not_Applicable1 TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_No_information TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_Unknown TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_Not_asked TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_askedbutunknown TEXT,
Eve_2_3_hours_before_going_to_bed_I_consumed_Not_applicable2 TEXT,
Eve_In_the_hour_before_sleep_my_bedtime_routine_included TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_protocol_deviation
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_protocol_deviation (
Study_ID TEXT,
Event_Name TEXT,
Was_there_a_protocol_deviation_recorded_for_this_participant TEXT,
Date_of_deviation TEXT,
Date_identified TEXT,
Deviation_description TEXT,
Deviation_Code__ TEXT,
Deviation_Code_Other TEXT,
Resulted_in_Adverse_Event TEXT,
Did_participant_continue_in_study TEXT,
Date_of_withdrawal TEXT,
Ethics_Reporting_required TEXT,
Ethics_report_date TEXT,
PI_signature TEXT,
Protocol_deviation_form_upload TEXT,
Complete TEXT,
updated_at TIMESTAMP)""")

# FU_report_calculations
mycursor.execute("""CREATE TABLE IF NOT EXISTS FU_report_calculations (
Study_ID TEXT,
Event_Name TEXT,
Pregnancy_survey_start_date DATE,
Pregnancy_survey_complete_date DATE,
Pregnancy_survey_duration_completion_minutes DECIMAL (10,3),
Pregnancy_survey_duration_minutes DECIMAL (10,3),
Sleep_diary_term_survey_start_date DATE,
Sleep_diary_term_survey_complete_date DATE,
Sleep_diary_term_survey_duration_completion_minutes DECIMAL (10,3),
Sleep_diary_term_survey_duration_minutes DECIMAL (10,3),
ASQ3_2_month_survey_start_date DATE,
ASQ3_2_month_survey_complete_date DATE,
ASQ3_2_month_survey_duration_completion_minutes DECIMAL (10,3),
ASQ3_2_month_survey_duration_minutes DECIMAL (10,3),
Maternal_Interim_2_month_survey_start_date DATE,
Maternal_Interim_2_month_survey_complete_date DATE,
Maternal_Interim_2_month_survey_duration_completion_minutes DECIMAL (10,3),
Maternal_Interim_2_month_survey_duration_minutes DECIMAL (10,3),
BISQ_R_2_month_survey_start_date DATE,
BISQ_R_2_month_survey_complete_date DATE,
BISQ_R_2_month_survey_duration_completion_minutes DECIMAL (10,3),
BISQ_R_2_month_survey_duration_minutes DECIMAL (10,3),
Sleep_diary_2_month_survey_start_date DATE,
Sleep_diary_2_month_survey_complete_date DATE,
Sleep_diary_2_month_survey_duration_completion_minutes DECIMAL (10,3),
Sleep_diary_2_month_survey_duration_minutes DECIMAL (10,3),
ASQ3_6_month_survey_start_date DATE,
ASQ3_6_month_survey_complete_date DATE,
ASQ3_6_month_survey_duration_completion_minutes DECIMAL (10,3),
ASQ3_6_month_survey_duration_minutes DECIMAL (10,3),
Maternal_Interim_6_month_survey_start_date DATE,
Maternal_Interim_6_month_survey_complete_date DATE,
Maternal_Interim_6_month_survey_duration_completion_minutes DECIMAL (10,3),
Maternal_Interim_6_month_survey_duration_minutes DECIMAL (10,3),
BISQ_R_6_month_survey_start_date DATE,
BISQ_R_6_month_survey_complete_date DATE,
BISQ_R_6_month_survey_duration_completion_minutes DECIMAL (10,3),
BISQ_R_6_month_survey_duration_minutes DECIMAL (10,3),
Sleep_diary_6_month_survey_start_date DATE,
Sleep_diary_6_month_survey_complete_date DATE,
Sleep_diary_6_month_survey_duration_completion_minutes DECIMAL (10,3),
Sleep_diary_6_month_survey_duration_minutes DECIMAL (10,3),
ASQ3_12_month_survey_start_date DATE,
ASQ3_12_month_survey_complete_date DATE,
ASQ3_12_month_survey_duration_completion_minutes DECIMAL (10,3),
ASQ3_12_month_survey_duration_minutes DECIMAL (10,3),
Maternal_Interim_12_month_survey_start_date DATE,
Maternal_Interim_12_month_survey_complete_date DATE,
Maternal_Interim_12_month_survey_duration_completion_minutes DECIMAL (10,3),
Maternal_Interim_12_month_survey_duration_minutes DECIMAL (10,3),
ASQ3_24_month_survey_start_date DATE,
ASQ3_24_month_survey_complete_date DATE,
ASQ3_24_month_survey_duration_completion_minutes DECIMAL (10,3),
ASQ3_24_month_survey_duration_minutes DECIMAL (10,3),
Maternal_Interim_24_month_survey_start_date DATE,
Maternal_Interim_24_month_survey_complete_date DATE,
Maternal_Interim_24_month_survey_duration_completion_minutes DECIMAL (10,3),
Maternal_Interim_24_month_survey_duration_minutes DECIMAL (10,3),
CSBS_Checklist_start_date DATE,
CSBS_Checklist_complete_date DATE,
CSBS_Checklist_duration_completion_minutes DECIMAL (10,3),
CSBS_Checklist_duration_minutes DECIMAL (10,3),
ITSEA_Questionnaire_start_date DATE,
ITSEA_Questionnaire_complete_date DATE,
ITSEA_Questionnaire_duration_completion_minutes DECIMAL (10,3),
ITSEA_Questionnaire_duration_minutes DECIMAL (10,3),
Year_2_survey_start_date DATE,
Year_2_survey_complete_date DATE,
Year_2_survey_duration_completion_minutes DECIMAL (10,3),
Year_2_survey_duration_minutes DECIMAL (10,3),
Complete TEXT,
updated_at TIMESTAMP)""")

## Import Data

### Core

In [270]:
# Identify the Columns for each Instrument
FU_Instruments = ["fu_core_details", "fu_contacts", "fu_growth", "fu_preg_surv", "fu_substudies", "fu_survival", "fu_asq3_2m", "fu_asq3_6m", "fu_asq3_12m", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_asq_score", "fu_itsea", "fu_itsea_score", "fu_csbs", "fu_csbs_score", "fu_ffm", "fu_bayley4", "fu_adverse_event", "fu_withdrawal", "fu_m_int_survey", "fu_y2_surv", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_sleep_diary", "fu_protocol_deviation", "fu_report_calculations"]

# View all the column numbers which link to "Complete?" heading which indicates the end of a instrument's data
Column_list = [2] # Starts from the very first row
for a in range (0 ,len(data[0])):
    temp = data[0][a]
    if temp == "Complete?": # This is the last column for the Instrument
        Column_list.append(a + 1)
        
event_name_list = ["Term", "cPNA1m", "cPNA2m", "cPNA3m", "cPNA3.5m", "cPNA6m", "cPNA12m", "cPNA22m", "cPNA24m", "Protocol Deviation"]  

# Manually create list for each event type that triggers an instrument
Term = ["fu_core_details", "fu_contacts", "fu_preg_surv", "fu_substudies", "fu_survival", "fu_asq_score", "fu_itsea_score", "fu_csbs_score", "fu_adverse_event", "fu_withdrawal", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_sleep_diary", "fu_report_calculations"]
cPNA1m = []
cPNA2m = ["fu_growth", "fu_asq3_2m", "fu_m_int_survey", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_sleep_diary"]
cPNA3m = []
cPNA3_5m = []
cPNA6m = ["fu_growth", "fu_asq3_6m", "fu_m_int_survey", "fu_actigraphy_child", "fu_actigraphy_maternal", "fu_salivary_swabs_collection", "fu_bisqr", "fu_sleep_diary"]
cPNA12m = ["fu_growth", "fu_asq3_12m", "fu_m_int_survey"]
cPNA22m = ["fu_itsea", "fu_csbs"]
cPNA24m = ["fu_growth", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_ffm", "fu_bayley4", "fu_m_int_survey", "fu_y2_surv"]
Protocol_Deviation = ["Protocol Deviation"]

event_name_mapping = [Term, cPNA1m, cPNA2m, cPNA3m, cPNA3_5m, cPNA6m, cPNA12m, cPNA22m, cPNA24m, Protocol_Deviation]
       
# The name of the filter for the repeat Instruments
event_name_list_repeatInstrument = ["Sleep Diary"]

# Create a dictionary with the Hospital Instruments / If it is a survey / Starting Column / Ending Column / Event to Instrument Mapping / Repeat_Instruments

# Tables are stored into the database based on whether they are a survey or not (Manually determined)
# 1 if Survey
# 0 if Not Survey

# Manually deduce whether or not the instrument should be stored in a survey structure
FU_Instruments_Survey = ["fu_asq3_2m", "fu_asq3_6m", "fu_asq3_12m", "fu_asq3_24m", "fu_asq3_27m", "fu_asq3_30m", "fu_asq3_33m", "fu_asq3_36m", "fu_itsea", "fu_csbs", "fu_m_int_survey", "fu_y2_surv", "fu_bisqr"]

# The datarows are filtered either based on event_name or repeat_instrument. This will display the right data row when the data should be present
# 1 if filtered by Event Name
# 0 if filtered by Repeat Instrument

# Data rows that need to be filtered by repeat_instruments rather than event_name
Repeat_Instruments = ["fu_sleep_diary"]

FU_Dictionary = {}
# Example screen: [0, 6, 41] -> instrument: "screen", not survey, first column: 6, last column: 41
for a in range (0, len(FU_Instruments)):
    table_name = FU_Instruments[a]
    
    temp_array = []
    # Check to see if the instrument is a survey
    if table_name in FU_Instruments_Survey:
         temp_array.append(1)
    else:
         temp_array.append(0)
         
    # First column and Last column of data
    temp_array.append(Column_list[a])
    temp_array.append(Column_list[a+1])
         
    # Check to see if the instrument needs to be filtered by Repeat Instrument
    if table_name in Repeat_Instruments:
        temp_array.append(1)
         # Append an array of the repeat_instrument strings that it need to be filtered on
        temp_list = []
        for b in range (0, len(Repeat_Instruments)):
            if Repeat_Instruments[b] == table_name:
                temp_list.append(event_name_list_repeatInstrument[b])
        temp_array.append(temp_list)
    else:
        temp_array.append(0)
        # Append an array of the event_names that need to be filtered on
        temp_list = []
        for b in range (0, len(event_name_mapping)):
            if table_name in event_name_mapping[b]:
                temp_list.append(event_name_list[b])
        temp_array.append(temp_list)
        
    # Create Dictionary
    FU_Dictionary[table_name] = temp_array

In [271]:
FU_Dictionary

{'fu_core_details': [0, 2, 27, 0, ['Term']],
 'fu_contacts': [0, 27, 67, 0, ['Term']],
 'fu_growth': [0, 67, 79, 0, ['cPNA2m', 'cPNA6m', 'cPNA12m', 'cPNA24m']],
 'fu_preg_surv': [0, 79, 101, 0, ['Term']],
 'fu_substudies': [0, 101, 110, 0, ['Term']],
 'fu_survival': [0, 110, 114, 0, ['Term']],
 'fu_asq3_2m': [1, 114, 178, 0, ['cPNA2m']],
 'fu_asq3_6m': [1, 178, 242, 0, ['cPNA6m']],
 'fu_asq3_12m': [1, 242, 306, 0, ['cPNA12m']],
 'fu_asq3_24m': [1, 306, 380, 0, ['cPNA24m']],
 'fu_asq3_27m': [1, 380, 415, 0, ['cPNA24m']],
 'fu_asq3_30m': [1, 415, 450, 0, ['cPNA24m']],
 'fu_asq3_33m': [1, 450, 485, 0, ['cPNA24m']],
 'fu_asq3_36m': [1, 485, 551, 0, ['cPNA24m']],
 'fu_asq_score': [0, 551, 641, 0, ['Term']],
 'fu_itsea': [1, 641, 832, 0, ['cPNA22m']],
 'fu_itsea_score': [0, 832, 884, 0, ['Term']],
 'fu_csbs': [1, 884, 917, 0, ['cPNA22m']],
 'fu_csbs_score': [0, 917, 938, 0, ['Term']],
 'fu_ffm': [0, 938, 942, 0, ['cPNA24m']],
 'fu_bayley4': [0, 942, 982, 0, ['cPNA24m']],
 'fu_adverse_event':

In [272]:
# Iterate through the dictionary of Instruments, Check to see if they are survey / Check to see what the rows are filtered on / Import to SQL

for table_name in FU_Dictionary:
    
    print(table_name)
    Survey_binary = FU_Dictionary[table_name][0]
    Column1 = FU_Dictionary[table_name][1]
    Column2 = FU_Dictionary[table_name][2]
    Repeat_Instrument_bin = FU_Dictionary[table_name][3]
    StringList = FU_Dictionary[table_name][4]
    
    
    # If the Instrument is NOT a survey
    if FU_Dictionary[table_name][0] == 0:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
            else:
                if event_name in StringList and repeat_instrument == "":
                    temp_data = data[a][0:2] + data[a][Column1:Column2]
                    import_data.append(temp_data)
                
        # Check to see if the final columne heading is "Comlete?"
        if data[0][Column2 - 1] != "Complete?":
            print("Error: Last Column was not 'Complete?' field")

        # Import the data into the correct table in the database
        import_database(table_name, import_data)
        print("{} imported".format(table_name))
        
    # If the Instrument is a survey
    if FU_Dictionary[table_name][0] == 1:
        
        # Retrieve the appropriate columns and rows
        import_data = []
        questions_data = []
        for a in range (0, len(data)):
            event_name = data[a][1]
            repeat_instrument = data[a][2]
            
            # Check to see if the data is filtered by repeat instrument or event_name
            if Repeat_Instrument_bin == 1:
                if repeat_instrument in StringList:
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)
            else:
                if event_name in StringList and repeat_instrument == "":
                    # Survey date / survey timestamp / Complete
                    temp_data = data[a][0:2] + data[a][Column1:Column1+1] + data[a][Column2-1:Column2] # Data that related to the survey
                    questions_data_row = data[a][Column1+1:Column2-1] # Survery data
                    import_data.append(temp_data)
                    questions_data.append(questions_data_row)

            # Gather the headings of questions of the survey
            questions_headings = data[0][Column1+1:Column2-1] 

        # Impot survey structured data into the database
        import_database_survey(table_name, import_data, questions_headings, questions_data)
        print("{} imported".format(table_name))

fu_core_details
fu_core_details imported
fu_contacts
fu_contacts imported
fu_growth
fu_growth imported
fu_preg_surv
fu_preg_surv imported
fu_substudies
fu_substudies imported
fu_survival
fu_survival imported
fu_asq3_2m
fu_asq3_2m imported
fu_asq3_6m
fu_asq3_6m imported
fu_asq3_12m
fu_asq3_12m imported
fu_asq3_24m
fu_asq3_24m imported
fu_asq3_27m
fu_asq3_27m imported
fu_asq3_30m
fu_asq3_30m imported
fu_asq3_33m
fu_asq3_33m imported
fu_asq3_36m
fu_asq3_36m imported
fu_asq_score
fu_asq_score imported
fu_itsea
fu_itsea imported
fu_itsea_score
fu_itsea_score imported
fu_csbs
fu_csbs imported
fu_csbs_score
fu_csbs_score imported
fu_ffm
fu_ffm imported
fu_bayley4
fu_bayley4 imported
fu_adverse_event
fu_adverse_event imported
fu_withdrawal
fu_withdrawal imported
fu_m_int_survey
fu_m_int_survey imported
fu_y2_surv
fu_y2_surv imported
fu_actigraphy_child
fu_actigraphy_child imported
fu_actigraphy_maternal
fu_actigraphy_maternal imported
fu_salivary_swabs_collection
fu_salivary_swabs_collection i

71

In [277]:
# # View all the headings of the the fields separated by the "Complete" field for different forms
# for a in range (Column1,Column2):
#     temp = data[0][a].replace(" ", "_").replace("(","").replace(")","").replace("?","").replace("=", "_").replace("+", "").replace(".", "").replace("/", "").replace("-", "_").replace(":", "")
#     print(a, temp)
    
#     if temp == "Complete":
#         print("--------------------------------------------------------------------------------")

# CIRCADIEM Hospital 5859

In [275]:
# Read the data file for import
filepath = "C:/Users/EPan/Documents/CIRCADIEM/Data/CIRCADIEM_Hospital_5859/CIRCADIEMHospital585_DATA_LABELS_2023-10-25_1743.csv"


data = []
with open(r'{}'.format(filepath)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append(row)
        
# To be finished when the CIRCADIEM Hospital 5859 database is finished being consolidated

In [276]:
# Release Memory
%reset -f